In [1]:
import string
import numpy as np
import math

In [2]:
text = 'otro/robert_frost.txt'

We need 3 elements:
1st the vector with the initial probbilities
2nd the matrix for the second state
3rd the tensor containing the probabilities of a word given a previous word

By advice of the teacher we'll try first to use dictionaries: so pi will be a dictionary, A1 will be a dictionary of dictionaries and A2 will be a dictionary of dictionaries of dictionaries

In [3]:
input_texts = []

for line in open(text):
    line = line.rstrip().lower()
    if line:
        line = line.translate(str.maketrans('','',string.punctuation))
        input_texts.append(line)


In [4]:
word_to_index_mapping = {}
indice = 1

for text in input_texts:
    text = text.split()
    for word in text:
        if word not in word_to_index_mapping:
            word_to_index_mapping[word]=indice
            indice+=1

In [5]:
word_to_index_mapping

{'two': 1,
 'roads': 2,
 'diverged': 3,
 'in': 4,
 'a': 5,
 'yellow': 6,
 'wood': 7,
 'and': 8,
 'sorry': 9,
 'i': 10,
 'could': 11,
 'not': 12,
 'travel': 13,
 'both': 14,
 'be': 15,
 'one': 16,
 'traveler': 17,
 'long': 18,
 'stood': 19,
 'looked': 20,
 'down': 21,
 'as': 22,
 'far': 23,
 'to': 24,
 'where': 25,
 'it': 26,
 'bent': 27,
 'the': 28,
 'undergrowth': 29,
 'then': 30,
 'took': 31,
 'other': 32,
 'just': 33,
 'fair': 34,
 'having': 35,
 'perhaps': 36,
 'better': 37,
 'claim': 38,
 'because': 39,
 'was': 40,
 'grassy': 41,
 'wanted': 42,
 'wear': 43,
 'though': 44,
 'for': 45,
 'that': 46,
 'passing': 47,
 'there': 48,
 'had': 49,
 'worn': 50,
 'them': 51,
 'really': 52,
 'about': 53,
 'same': 54,
 'morning': 55,
 'equally': 56,
 'lay': 57,
 'leaves': 58,
 'no': 59,
 'step': 60,
 'trodden': 61,
 'black': 62,
 'oh': 63,
 'kept': 64,
 'first': 65,
 'another': 66,
 'day': 67,
 'yet': 68,
 'knowing': 69,
 'how': 70,
 'way': 71,
 'leads': 72,
 'on': 73,
 'doubted': 74,
 'if': 75

In [6]:
dataset_as_int = []

for line in input_texts:
    new_line = []
    line = line.split()
    for word in line:
        new_line.append(word_to_index_mapping[word])
    dataset_as_int.append(new_line)

In [7]:
dataset_as_int

[[1, 2, 3, 4, 5, 6, 7],
 [8, 9, 10, 11, 12, 13, 14],
 [8, 15, 16, 17, 18, 10, 19],
 [8, 20, 21, 16, 22, 23, 22, 10, 11],
 [24, 25, 26, 27, 4, 28, 29],
 [30, 31, 28, 32, 22, 33, 22, 34],
 [8, 35, 36, 28, 37, 38],
 [39, 26, 40, 41, 8, 42, 43],
 [44, 22, 45, 46, 28, 47, 48],
 [49, 50, 51, 52, 53, 28, 54],
 [8, 14, 46, 55, 56, 57],
 [4, 58, 59, 60, 49, 61, 62],
 [63, 10, 64, 28, 65, 45, 66, 67],
 [68, 69, 70, 71, 72, 73, 24, 71],
 [10, 74, 75, 10, 76, 77, 78, 79],
 [10, 80, 15, 81, 82, 83, 5, 84],
 [85, 86, 8, 86, 87],
 [1, 2, 3, 4, 5, 7, 8, 10],
 [10, 31, 28, 16, 88, 89, 90],
 [8, 46, 91, 92, 93, 28, 94],
 [95, 96, 97, 98, 10, 99, 10, 100],
 [101, 102, 103, 4, 28, 104, 44],
 [105, 106, 12, 107, 108, 109, 110],
 [24, 111, 101, 96, 112, 113, 83, 114],
 [115, 116, 117, 118, 99, 26, 119],
 [24, 120, 121, 5, 122, 123],
 [124, 28, 96, 8, 125, 126],
 [28, 127, 128, 129, 28, 130],
 [105, 131, 101, 132, 133, 5, 134],
 [24, 135, 75, 48, 103, 136, 137],
 [28, 138, 32, 139, 28, 140],
 [129, 141, 142,

First we create the model

In [8]:
pi = {}
A1 = {}
A2 = {}

# Calculate pi probabilities
for line in input_texts:
    line = line.split()
    for index, word in enumerate(line):
        if index == 0 and word not in pi.keys():
            pi[word] = 1
        elif index == 0 and word in pi.keys():
            pi[word] += 1

for word_value in pi.keys():
    pi[word_value] = pi[word_value]/len(pi)


# Calculate A1 probabilities
for line in input_texts:
    line = line.split()
    for index, word in enumerate(line):
        if index == 1:
            if line[index-1] not in A1.keys():
                A1[line[index-1]] = {}
            if word not in A1[line[index-1]].keys():
                A1[line[index-1]][line[index]] = 0
            if word in A1[line[index-1]].keys():
                A1[line[index-1]][word] +=1
        else:
            pass
for state in A1.keys():
    total = len(A1[state])
    for word in A1[state]:
        A1[state][word] = A1[state][word]/total

# Calculate A2 probabilities
for line in input_texts:
    line = line.split()
    for index, word in enumerate(line):
        if index >= 2:
            if line[index-2] not in A2.keys():
                A2[line[index-2]] = {}
            if line[index-1] not in A2[line[index-2]].keys():
                A2[line[index-2]][line[index-1]] = {}
            if line[index] not in A2[line[index-2]][line[index-1]]:
                A2[line[index-2]][line[index-1]][word] = 1
            else:
                A2[line[index-2]][line[index-1]][word] += 1
        else:
            pass

In [9]:
pi

{'two': 0.02622950819672131,
 'and': 0.42295081967213116,
 'to': 0.16393442622950818,
 'then': 0.03934426229508197,
 'because': 0.003278688524590164,
 'though': 0.022950819672131147,
 'had': 0.013114754098360656,
 'in': 0.09508196721311475,
 'oh': 0.013114754098360656,
 'yet': 0.009836065573770493,
 'i': 0.38688524590163936,
 'somewhere': 0.003278688524590164,
 'whose': 0.006557377049180328,
 'his': 0.022950819672131147,
 'he': 0.11147540983606558,
 'my': 0.022950819672131147,
 'between': 0.009836065573770493,
 'the': 0.26885245901639343,
 'of': 0.09508196721311475,
 'but': 0.16721311475409836,
 'some': 0.01639344262295082,
 'from': 0.03278688524590164,
 'is': 0.01639344262295082,
 'natures': 0.003278688524590164,
 'her': 0.006557377049180328,
 'so': 0.04262295081967213,
 'nothing': 0.006557377049180328,
 'when': 0.029508196721311476,
 'came': 0.003278688524590164,
 'one': 0.036065573770491806,
 'proclaimed': 0.003278688524590164,
 'smoothlaid': 0.003278688524590164,
 'half': 0.0098360

In [10]:
A1

{'two': {'roads': 0.2857142857142857,
  'miles': 0.14285714285714285,
  'oldbelievers': 0.14285714285714285,
  'winds': 0.14285714285714285,
  'weeks': 0.14285714285714285,
  'of': 0.14285714285714285,
  'at': 0.14285714285714285},
 'and': {'sorry': 0.019417475728155338,
  'be': 0.009708737864077669,
  'looked': 0.009708737864077669,
  'having': 0.009708737864077669,
  'both': 0.019417475728155338,
  'that': 0.019417475728155338,
  'miles': 0.019417475728155338,
  'would': 0.009708737864077669,
  'dropped': 0.009708737864077669,
  'further': 0.009708737864077669,
  'when': 0.009708737864077669,
  'tell': 0.02912621359223301,
  'the': 0.038834951456310676,
  'caught': 0.009708737864077669,
  'put': 0.019417475728155338,
  'threw': 0.009708737864077669,
  'birds': 0.009708737864077669,
  'suddenly': 0.009708737864077669,
  'scurf': 0.009708737864077669,
  'since': 0.019417475728155338,
  'whats': 0.009708737864077669,
  'many': 0.009708737864077669,
  'blew': 0.009708737864077669,
  'sta

In [11]:
A2

{'two': {'roads': {'diverged': 2},
  'miles': {'it': 1},
  'towns': {'fighting': 1},
  'converging': {'slides': 1},
  'or': {'three': 1},
  'oldbelievers': {'they': 1},
  'legs': {'like': 1},
  'footsteps': {'for': 1},
  'village': {'cultures': 1},
  'winds': {'would': 1},
  'weeks': {'since': 1},
  'of': {'you': 1},
  'as': {'good': 1},
  'at': {'a': 1}},
 'roads': {'diverged': {'in': 2}},
 'diverged': {'in': {'a': 2}},
 'in': {'a': {'yellow': 1,
   'wood': 1,
   'window': 1,
   'packing': 1,
   'byroad': 1,
   'family': 1,
   'new': 1,
   'row': 1,
   'time': 1,
   'town': 1,
   'book': 1,
   'smother': 1,
   'glass': 2},
  'the': {'undergrowth': 1,
   'village': 1,
   'stars': 1,
   'confusion': 1,
   'excited': 1,
   'scene': 1,
   'river': 1,
   'cold': 2,
   'town': 1,
   'warrant': 1,
   'stable': 1,
   'huse': 1,
   'tail': 1,
   'dark': 2,
   'mountain': 1,
   'box': 1,
   'night': 1,
   'bedroom': 1,
   'cellar': 7,
   'joints': 1,
   'light': 1,
   'room': 1,
   'attic': 2,
